# Part 1: Importing Necessary Libraries

In [ ]:
# Gemini
import google.generativeai as genai
from IPython.display import Markdown

# System and Environment
import os   
from dotenv import load_dotenv

# For Dataframes
import pandas as pd
import numpy as np
import random

# Data visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
from spotipy.oauth2 import SpotifyOAuth
import webbrowser
from json.decoder import JSONDecodeError

# pytest
import pytest
import unittest
from unittest.mock import patch, Mock

In [ ]:
# Load Environment Variables and Configure Gemini
load_dotenv()
    
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
REDIRECT_URI = os.getenv("REDIRECT_URI")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

# Part 2: Logging in With Authentication

In [ ]:
# Create App With User Authentication

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
client_secret=CLIENT_SECRET,
redirect_uri=REDIRECT_URI,
scope="user-library-read user-top-read"))

In [ ]:
# Get top 50 tracks from short term
top_tracks = sp.current_user_top_tracks(limit=50, time_range='short_term')

track_info = []
for item in top_tracks['items']:
    track_name = item["name"]
    artist_name = item["artists"][0]["name"]

    try:
        artist = sp.search(q=artist_name, type="artist")["artists"]["items"][0]
        genres = artist["genres"]
    except IndexError:
        genres = []

    track_info.append({
        "track": track_name,
        "artist": artist_name,
        "genres": genres
    })

df = pd.DataFrame(track_info)
df.head()

In [ ]:
vibe_model = genai.GenerativeModel('gemini-1.5-flash')

def get_top_vibes(track_list, top_n=3):
    """Analyze a list of tracks and return dominant vibes"""
    # Prepare track information for Gemini
    track_summary = "\n".join([
        f"- {track['name']} by {track['artists'][0]['name']}" 
        for track in track_list['items']
    ])
    
    prompt = f"""
    Analyze these songs from my listening history and identify the dominant vibes:
    {track_summary}
    
    Possible vibe categories (choose up to {top_n} dominant ones):
    - Chill 
    - Energetic 
    - Melancholy
    - Romantic
    - Confident
    - Nostalgic
    - Artsy
    - Dark
    - Rage
    - Futuristic
    - Party
    - Ambient
    - Spiritual
    - Dreamy
    - Rebellious
    - Carefree
    - Classy
    - Cinematic
    - Theatrical
    - Alternative
    
    Provide only the {top_n} most dominant vibe categories as a comma-separated list (no explanations).
    Example: "Energetic/Upbeat, Chill/Relaxed, Emotional/Introspective"
    """
    
    try:
        response = vibe_model.generate_content(prompt)
        return [vibe.strip() for vibe in response.text.split(",")[:top_n]]
    except Exception as e:
        print(f"Error analyzing vibes: {e}")
        return ["Unknown"]

# Get top tracks and analyze overall vibes
top_tracks = sp.current_user_top_tracks(limit=50, time_range='short_term')
dominant_vibes = get_top_vibes(top_tracks)

print("\nYour dominant music vibes:")
for i, vibe in enumerate(dominant_vibes, 1):
    print(f"{i}. {vibe}")